In [1]:
import torch
from transformers import AutoTokenizer, BartForConditionalGeneration
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

# Model

### Ubuntu 22.04 LTS CUDA

In [ ]:
model_dir = "weights/alaggung-bart-r3f/"
model_dir2 = "weights/kobart-summ/"
max_length = 64
num_beams = 5
length_penalty = 1.2

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = BartForConditionalGeneration.from_pretrained(model_dir)

tokenizer_full = PreTrainedTokenizerFast.from_pretrained(model_dir2)
model_full = BartForConditionalGeneration.from_pretrained(model_dir2)

### Apple Silicon Mps

In [2]:
model_dir = "weights/alaggung-bart-r3f/"
model_dir2 = "weights/kobart-summ/"
max_length = 64
num_beams = 5
length_penalty = 1.2

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = BartForConditionalGeneration.from_pretrained(model_dir)

tokenizer_full = PreTrainedTokenizerFast.from_pretrained(model_dir2)
model_full = BartForConditionalGeneration.from_pretrained(model_dir2)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# Inference (diary)

In [3]:
text = """
1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다.
방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다.
국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다.
동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다.
확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다.
오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다.
17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다.
비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다.
부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 광주 5130명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다.
집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다.
한편 전날 하루 선별진료소에서 이뤄진 검사는 70만8763건으로 검사 양성률은 40.5%다. 양성률이 40%를 넘은 것은 이번이 처음이다. 확산세가 계속 거세질 수 있다는 얘기다.
이날 0시 기준 신규 확진자는 13만8993명이었다. 이틀 연속 13만명대를 이어갔다.
"""

In [11]:
text = text.replace('\n', ' ')

raw_input_ids = tokenizer_full.encode(text)
input_ids = [tokenizer_full.bos_token_id] + raw_input_ids + [tokenizer_full.eos_token_id]

summary_ids = model_full.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
tokenizer_full.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

# Inference (dialogue)

In [8]:
dialogue = [
    "얌 수인이랑 5/21에 보기로 했는데 너도 시간 되면 올래?", 
    "오 그래?? 좋지 좋지", 
    "되게 만들어야지", 
    "오 오키오키!", 
    "아직 어디서 볼지는 안 정한거지?", 
    "웅웅 날짜밖에 안 정햇어",
    "부발역은",
    "어디가 좋은데??",
    "안되나요",
    "ㅋㅋㅋㅋ",
    "ㅋㅋㅋㅋㅋ에바여 할거없음"
]

In [9]:
# model.eval()
inputs = tokenizer("[BOS]" + "[SEP]".join(dialogue) + "[EOS]", return_tensors="pt")
outputs = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    num_beams=num_beams,
    length_penalty=length_penalty,
    max_length=max_length,
    use_cache=True,
)

In [10]:
summarization = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summarization)

수인과 5월 21일에 부발역에서 만나기로 했다.
